<a href="https://colab.research.google.com/github/sayanbanerjee32/learning-equality-curriculum-recommendations/blob/main/lecr_SentTrans_data_prep_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

!pip install -Uq datasets
!pip install -Uq transformers
!pip install -Uq sentencepiece 
!pip install -Uq evaluate
!pip install -Uq pynvml
!pip install -Uq sentence_transformers
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB

In [2]:
# # This get the RAPIDS-Colab install files and test check your GPU.  Run cells 1 and 2 only.
# # Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/env-check.py

In [3]:
# # This will update the Colab environment and restart the kernel.
# !bash rapidsai-csp-utils/colab/update_gcc.sh
# import os
# os._exit(00)

In [4]:
# ## Installing CondaColab.  This will restart your kernel again
# import condacolab
# condacolab.install()

In [5]:
# # you can now run the rest of the cells as normal
# import condacolab
# condacolab.check()

In [6]:
# # Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# # The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
# # The <packages> option are default blank or 'core'.  By default, we install RAPIDSAI and BlazingSQL.  The 'core' option will install only RAPIDSAI and not include BlazingSQL, 
# !python rapidsai-csp-utils/colab/install_rapids.py nightly
# import os
# os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
# os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
# os.environ['CONDA_PREFIX'] = '/usr/local'

In [7]:
from fastkaggle import *
import os
from pathlib import Path
import pandas as pd
import numpy as np # linear algebra
from tqdm.auto import tqdm
import gc

import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from datasets import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


In [8]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
comp = 'learning-equality-curriculum-recommendations'
if is_colab:
    model_save_path = Path('/content/'+comp+'_out/models')
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')

In [9]:
# import colab libraries
if is_colab:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [11]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [12]:
path = setup_comp(comp)
path

100%|██████████| 254M/254M [00:13<00:00, 19.5MB/s]


Path('learning-equality-curriculum-recommendations')

### read all input files

In [13]:
# load the data into pandas dataframes
topics_df = pd.read_csv(path / "topics.csv", index_col=0).fillna({"title": "", "description": ""})
content_df = pd.read_csv(path / "content.csv", index_col=0).fillna("")
correlations_df = pd.read_csv(path / "correlations.csv", index_col=0)

In [14]:
print(topics_df.shape, correlations_df.shape, content_df.shape)
correlations_df.head()

(76972, 8) (61517, 1) (154047, 7)


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4


In [15]:
def process_correlations(correlations_df, label_value):
    correlations = correlations_df.copy()
    correlations.content_ids = correlations.content_ids.str.split()
    correlations = correlations.explode("content_ids").rename(columns={"content_ids": "content_id"}).reset_index()
    correlations['label'] = label_value
    return correlations
correlations = process_correlations(correlations_df, 1)
correlations.head()

,topic_id,content_id,label
0,t_00004da3a1b2,c_1108dd0c7a5d,1
1,t_00004da3a1b2,c_376c5a8eb028,1
2,t_00004da3a1b2,c_5bc0e1e2cba0,1
3,t_00004da3a1b2,c_76231f9d0b5e,1
4,t_00068291e9a4,c_639ea2ef9c95,1


In [16]:
content_df.head()

,title,description,kind,text,language,copyright_holder,license
id,,,,,,,
c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,,es,,
c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,,it,,
c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,,es,,
c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA


#### Negative Samples

In [17]:
### sample records for colab only - test bed
if is_colab:
    correlations_df = correlations_df.sample(frac=0.1)
    sampled_topic_ids = correlations_df.reset_index().topic_id.to_list()
    correlations = correlations.query('topic_id in @sampled_topic_ids')
    sampled_content_ids = correlations.content_id.to_list()
    tmp_content_df = content_df.query('id in @sampled_content_ids')
    negative_content_df = content_df.query('id not in @sampled_content_ids').sample(frac=0.1)
    content_df = pd.concat([tmp_content_df,negative_content_df], axis = 0)
correlations_df.shape, len(sampled_topic_ids), correlations.shape, content_df.shape

((6152, 1), 6152, (28309, 3), (38096, 7))

### Find all Similer contents for each topic

#### generate content embedding

In [18]:
# create data set
content_dataset = Dataset.from_pandas(content_df)

In [19]:
# TODO - need to find a way to use text
def concatenate_text(examples):
    return {
        "concat_text": (examples["title"]
        + " \n "
        + examples["description"].replace('\n',' ').strip()
        + " \n "
        + examples["text"].replace(examples["title"],'').strip().split("\n")[0][:100]
        ).strip()
    }
content_dataset = content_dataset.map(concatenate_text)

Map:   0%|          | 0/38096 [00:00<?, ? examples/s]

In [20]:
# model_nm = 'microsoft/mdeberta-v3-base'
# MAX_LEN = 256
model_nm = 'all-MiniLM-L6-v2'
MAX_LEN = 256
batch_size = 64
SELECT_TOP_N = 25

In [21]:
# Load models
# model = AutoModel.from_pretrained(model_nm)
# model.eval()
# model.to(device)

# tokenizer = AutoTokenizer.from_pretrained(model_nm)
## sentence transformer
model = SentenceTransformer(model_nm)

In [22]:
# helper function cleaning GPU memory
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [23]:
# helper funcion for embedding and pooling
# def cls_pooling(model_output):
#     return model_output.last_hidden_state[:, 0]

# def mean_pooling(model_output, attention_mask):
#     # Extract the token embeddings
#     token_embeddings = model_output[0]
#     # Compute the attention mask
#     input_mask_expanded = (attention_mask
#                            .unsqueeze(-1)
#                            .expand(token_embeddings.size())
#                            .float())
#     # Sum the embeddings, but ignore masked tokens
#     sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
#     sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#     # Return the average as a single vector
#     return sum_embeddings / sum_mask

# def get_embeddings(text_list):
#     encoded_input = tokenizer(
#         text_list, padding=True, truncation=True, return_tensors="pt",
#         max_length=MAX_LEN
#     )
#     encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
#     model_output = model(**encoded_input)
#     # pooled_embeds = mean_pooling(model_output, encoded_input["attention_mask"])
#     return cls_pooling(model_output)

In [24]:
# test embedding
# embedding = get_embeddings(content_dataset["concat_text"][0])
embedding = model.encode(content_dataset["concat_text"][0:2])
embedding.shape

(2, 384)

In [25]:
report_gpu()
content_embeddings_dataset = content_dataset.map( 
    lambda x: {"embeddings": (model.encode(x["concat_text"])
                                # .detach().cpu().numpy()[0]
                              )
                },
batched=True)

GPU:0
process       5026 uses     1126.000 MB GPU memory


Map:   0%|          | 0/38096 [00:00<?, ? examples/s]

In [26]:
report_gpu()
content_embeddings_dataset.add_faiss_index(column="embeddings")

GPU:0
process       5026 uses     1422.000 MB GPU memory


  0%|          | 0/39 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'description', 'kind', 'text', 'language', 'copyright_holder', 'license', 'id', 'concat_text', 'embeddings'],
    num_rows: 38096
})

#### generate topic embedding

In [27]:
topic_ids = correlations_df.reset_index()['topic_id'].tolist()
_topics_df = topics_df.query(f'id in {topic_ids}')
_topics_df.head()

,title,description,channel,category,level,language,parent,has_content
id,,,,,,,,
t_00102869fbcb,Triangles and polygons,Learning outcomes: students must be able to so...,a91e32,aligned,3,en,t_039cecc12bb8,True
t_0020bde404c1,6.4.2 Writing of the electron arrangement of i...,Materials in this folder have been reviewed by...,c7ca13,aligned,6,en,t_ede9830986b2,True
t_00253e2bc084,Auto-tudo,"Autovetores, autovalores, autoespaços! Não par...",8e286a,source,4,pt,t_8a2a6d996ab4,True
t_002f563fc457,Les soins envers la nature,,c36745,supplemental,2,fr,t_9e969f10c821,True
t_002ff3ee9cb1,Acessórios e equipamentos para amamentação,,ffbd13,source,3,pt,t_5c306089d456,True


In [28]:
# create data set
topic_dataset = Dataset.from_pandas(_topics_df)
topic_dataset

Dataset({
    features: ['title', 'description', 'channel', 'category', 'level', 'language', 'parent', 'has_content', 'id'],
    num_rows: 6152
})

In [29]:
# concatenate title and description
def concatenate_topic_text(examples):
    return {
        "concat_text": (examples["title"]
        + " \n "
        + examples["description"].replace('\n',' ').strip()
        ).strip()
    }
topic_dataset = topic_dataset.map(concatenate_topic_text)

Map:   0%|          | 0/6152 [00:00<?, ? examples/s]

In [30]:
# test embedding
report_gpu()
# embedding = get_embeddings(topic_dataset["concat_text"][0])
embedding = model.encode(topic_dataset["concat_text"][0])
embedding.shape

GPU:0
process       5026 uses     1124.000 MB GPU memory


(384,)

In [31]:
report_gpu()
topic_embeddings_dataset = topic_dataset.map(
    # lambda x: {"embeddings": get_embeddings(x["concat_text"]).detach().cpu().numpy()[0]}
    lambda x: {"embeddings": model.encode(x["concat_text"])},
    batched=True
)

GPU:0
process       5026 uses     1126.000 MB GPU memory


Map:   0%|          | 0/6152 [00:00<?, ? examples/s]

### Similarity Search

In [33]:
## test with a single search
report_gpu()
scores, samples = content_embeddings_dataset.get_nearest_examples(
    "embeddings",
     np.array(topic_embeddings_dataset[0]['embeddings'], dtype=np.float32),
     k=SELECT_TOP_N
)
topic_embeddings_dataset[0]['concat_text'],samples['concat_text']

GPU:0
process       5026 uses     1482.000 MB GPU memory


('Triangles and polygons \n Learning outcomes: students must be able to solve problems involving angles, triangles and polygons including: types of triangles, calculate the interior and exterior angles of a triangle, different types of polygons, interior angles and sides of a convex polygon, the size and exterior angle of any convex polygon.',
 ['Solving Problems Involving Polygons \n In this lesson, you will learn how we can use the exterior angles of polygon formula to solve problems. \n Mary Chagwa, Blantyre',
  "Triangle exterior angle example \n Learn about the angles in a triangle and a little about exterior angles being the sum of the remote interior angles. \n What I want to do now is just a series of problems that really make sure that we know what we're doi",
  'Finding angle measures using triangles \n Use your knowledge about angles to find missing angle measures in various complex situations.',
  'Using the Formula to Solve Interior Angles of Convex Polygon \n In the previ

In [34]:
## using range search in order to get the indices
faiss_index = content_embeddings_dataset.get_index('embeddings').faiss_index
limits, distances, indices = faiss_index.range_search(x=np.array(topic_embeddings_dataset['embeddings'], dtype=np.float32).reshape(len(topic_embeddings_dataset),-1),thresh=0.97)
len(limits), len(indices), len(distances)

(6153, 1262270, 1262270)

In [36]:
neg_correlation_dict = {}
for i, j, t_id in zip(limits[:-1], limits[1:], topic_embeddings_dataset['id']):
    sorted_indices = [x for _, x in sorted(zip(distances[i:j], indices[i:j]))]
    neg_correlation_dict[t_id] = ' '.join([content_embeddings_dataset[int(ind)]['id'] for ind in sorted_indices[:SELECT_TOP_N]])

neg_correlation_df = pd.DataFrame(neg_correlation_dict.items(), columns = ['topic_id','content_ids']).set_index('topic_id')
print(neg_correlation_df.info(null_counts = True))
neg_correlation_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 6152 entries, t_00102869fbcb to t_fffe14f1be1e
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   content_ids  6152 non-null   object
dtypes: object(1)
memory usage: 96.1+ KB
None


<ipython-input-36-db9d2222b6aa>:7: FutureWarning: null_counts is deprecated. Use show_counts instead
  print(neg_correlation_df.info(null_counts = True))


,content_ids
topic_id,
t_00102869fbcb,c_ebde478f5297 c_af02c8d450c5 c_cb261bd5542c c...
t_0020bde404c1,c_2b0960ad94f2 c_d4cfca5992fa c_0578509aeb9f c...
t_00253e2bc084,c_abd8a9430cf4 c_50194cccd144 c_f72a112c90b4 c...
t_002f563fc457,c_5d5788ebeb4b c_79fcd5e17b5c c_c08fd92b1569 c...
t_002ff3ee9cb1,c_bef87430eb33 c_39b57b80ecc9 c_3acb11a7f09d c...


In [37]:
neg_correlations = process_correlations(neg_correlation_df, 0)
neg_correlations.head()

,topic_id,content_id,label
0,t_00102869fbcb,c_ebde478f5297,0
1,t_00102869fbcb,c_af02c8d450c5,0
2,t_00102869fbcb,c_cb261bd5542c,0
3,t_00102869fbcb,c_bebd175342f5,0
4,t_00102869fbcb,c_9addd7a27711,0


In [51]:
## contact positive and neagtive correlations
pos_neg_corr = pd.concat([correlations, neg_correlations])
#de-duplicate
pos_neg_corr = pos_neg_corr.sort_values(['topic_id','content_id','label'], ascending = [True, True, False]).drop_duplicates(['topic_id','content_id'])
pos_neg_corr.label.value_counts()

0    99630
1    28309
Name: label, dtype: int64

In [ ]:
# ### batch search
# report_gpu()
# _, samples = content_embeddings_dataset.get_nearest_examples_batch(
#         "embeddings",
#         np.array(topic_embeddings_dataset['embeddings'], dtype=np.float32),
#         k=SELECT_TOP_N
#      )
# (len(samples),
#  topic_embeddings_dataset[1]['concat_text'],
#  samples[1]['concat_text'])

GPU:0
process       4596 uses     1115.000 MB GPU memory


(6152,
 'يحدّد بعض قواعد الاشتقاق: يجد مشتقة اقتران باستخدام قاعدة السلسلة \n 5acb7211ecf6d9049f561089',
 ['يجد قاعدة الاقتران التمرين \n 5aded7228f9c1404a0d1e45f \n ما هي قاعدة الاقتران الموضح في الشكل :',
  'يحل معادلات مثلثية (الحل الأولي والحل العام) جبريا التمرين \n 5acc8c7becf6d904a02890da \n الحل الأولي للمعادلة قا س - ظا٢س = -١ هو ؟',
  'يتعرّف قوانين القوى (الأسس) الصحيحة غير السالبة والسالبة لأعداد حقيقيّة التمرين \n 5a4c84417dd197090857f5cd \n جد قيمة المقدار:',
  'يقرأ التمثيل البياني بالقطاعات الدائرية و يفسرها التمرين \n 5a58a2f77dd197717bd5df19 \n الجدول المجاور يمثل مساحات خمس قطع أراض',
  'القواعد \n  \n حدّد اسم الحرف المشبّه بالفعل وخبره في ما يلي: "قَرَأْتُ أَنَّ الأُسْتراليّينَ أَتَوْا مِنْ أَنْدوني',
  'يستعمل مبرهنة عكس فيثاغورس في اثبات مثلث قائم التمرين \n 5a4c84577dd19709085801ca \n يمثل لشكل أدناه منطاداً هوائياً. أوجد ارتفاعه عن سطح الأرض.',
  'يطبق قوانين الاحتمالات في مسائل حياتية التمرين \n 5ae17c0f6b9064043c652208 \n إذا كان احتمال قبول طالب في الجامعة ي

## Merge all DF

In [52]:
topic_content_details = pos_neg_corr.merge(topics_df,
                                           left_on = 'topic_id',
                                        right_on = 'id').merge(content_df,
                                                                  left_on = 'content_id',
                                                                right_on = 'id',
                                                               suffixes=('_topic', '_content'))
topic_content_details.head()

,topic_id,content_id,label,title_topic,description_topic,channel,category,level,language_topic,parent,has_content,title_content,description_content,kind,text,language_content,copyright_holder,license
0,t_00102869fbcb,c_005e793bd0c5,1,Triangles and polygons,Learning outcomes: students must be able to so...,a91e32,aligned,3,en,t_039cecc12bb8,True,Regular and Irregular Polygons,,html5,,en,,
1,t_2b0ffdf1e83c,c_005e793bd0c5,0,6.4.2 Area of regular polygons,,5d4a02,aligned,4,en,t_a02e3f30d367,True,Regular and Irregular Polygons,,html5,,en,,
2,t_4816635c284c,c_005e793bd0c5,0,Polygons,"Review your knowledge about triangles, quadril...",0c929f,source,4,sw,t_a38c139feb88,True,Regular and Irregular Polygons,,html5,,en,,
3,t_4a04c31e2abf,c_005e793bd0c5,0,Area of figures having irregular shapes,,8bbad6,supplemental,4,en,t_08ff7d3577fd,True,Regular and Irregular Polygons,,html5,,en,,
4,t_7a6c9dc992b6,c_005e793bd0c5,0,Construção de hexágono regular,,8e286a,source,4,pt,t_984fe2814e0a,True,Regular and Irregular Polygons,,html5,,en,,


In [53]:
### Any case where has_content = False
sum(~topic_content_details.has_content)

0

In [54]:
## check thr proportion of positive and negative
topic_content_details.label.value_counts()

0    99478
1    28309
Name: label, dtype: int64

In [56]:
### dump final dataframe
model_save_path.mkdir(parents=True, exist_ok=True)
topic_content_details.to_pickle(model_save_path / 'topic_content_details.pkl')

In [ ]:
if not iskaggle:
    push_notebook('saan', comp,
                  title='LECR: Sentence Transformer - Data prep',
                  file='/content/drive/MyDrive/Colab Notebooks/lecr_SentTrans_data_prep_baseline.ipynb',
                  competition=comp, private=False, gpu=True)